In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
import wget
from pathlib import Path
import mne
import colorcet as cc
import holoviews as hv
from holoviews.plotting.links import RangeToolLink
from holoviews.operation.datashader import rasterize
from holoviews.operation.downsample import downsample1d
from bokeh.models import HoverTool
import panel as pn

pn.extension()
hv.extension('bokeh')

np.random.seed(0)


data_url = 'https://physionet.org/files/eegmmidb/1.0.0/S001/S001R04.edf'
output_directory = Path('./data')

output_directory.mkdir(parents=True, exist_ok=True)
data_path = output_directory / Path(data_url).name
if not data_path.exists():
    data_path = wget.download(data_url, out=str(data_path))
    
    
raw = mne.io.read_raw_edf(data_path, preload=True)

raw.set_eeg_reference("average")

raw.rename_channels(lambda s: s.strip("."));

df = raw.to_data_frame() # TODO: fix rangetool for time_format='datetime'
df.set_index('time', inplace=True) 
df.head()

# Viz

amplitude_dim = hv.Dimension("amplitude", unit="µV")
time_dim = hv.Dimension("time", unit="s") # match the index name in the df

curves = {}
for channel_name, channel_data in df.items():
    
    curve = hv.Curve(df, kdims=[time_dim], vdims=[channel_name], group="EEG", label=channel_name)
    
    # TODO: Without the redim, downsample1d errors. But with, it prevents common index slice optimization. :(
    # curve = curve.redim(**{str(channel_name): amplitude_dim})

    curve = curve.opts(
        subcoordinate_y=True,
        subcoordinate_scale=2,
        color="black",
        line_width=1,
        tools=["hover"],
        hover_tooltips=[
            ("type", "$group"),
            ("channel", "$label"),
            ("time"),  # TODO: '@time{%H:%M:%S.%3N}'),
            ("amplitude"),
        ],
    )
    curves[channel_name] = curve
    
curves_overlay = hv.Overlay(curves, kdims="channel").opts(
    ylabel="channel",
    show_legend=False,
    padding=0,
    min_height=500,
    responsive=True,
    shared_axes=False,
    framewise=False,
)

# curves_overlay = downsample1d(curves_overlay, algorithm='minmax-lttb')

# minimap

channels = df.columns
time = df.index.values

y_positions = range(len(channels))
yticks = [(i, ich) for i, ich in enumerate(channels)]
z_data = zscore(df, axis=0).T
minimap = rasterize(hv.Image((time, y_positions, z_data), ["Time", "Channel"], "amplitude"))
minimap = minimap.opts(
    cmap="RdBu_r",
    colorbar=False,
    xlabel='',
    alpha=0.5,
    yticks=[yticks[0], yticks[-1]],
    toolbar='disable',
    height=120,
    responsive=True,
    # default_tools=[],
    cnorm='eq_hist'
    )

RangeToolLink(minimap, curves_overlay, axes=["x", "y"],
              boundsx=(0, time[len(time)//3]) # limit the initial x-range of the minimap
             )

layout = (curves_overlay + minimap).cols(1)
layout

In [ ]:
print(curves_overlay)